In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
physical_devices = tf.config.list_physical_devices('GPU')
sns.set()
Candledf=pd.read_csv('../result.dat',low_memory=False)
Candledf['ndatetime']=pd.to_datetime(Candledf['ndatetime'],format='%Y-%m-%d %H:%M:%S.%f')
print(Candledf.tail())

                ndatetime   open   high    low  close  volume
10107 2021-11-04 11:33:58  17122  17131  17107  17107   12000
10108 2021-11-04 12:04:07  17106  17130  17088  17091   12000
10109 2021-11-04 12:36:30  17091  17113  17090  17091   12000
10110 2021-11-04 13:10:55  17091  17103  17075  17075   12000
10111 2021-11-04 13:34:30  17075  17076  17050  17075   11866


In [2]:
def normalize(df):
    norm = df.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
    return norm

In [3]:
Candledf['Y'] = Candledf.close.shift(-1)
print(Candledf.Y.tail(5))
Candledf.iloc[:,1:5] = normalize(Candledf.iloc[:,1:4])
Candledf.head()

10107    17091.0
10108    17091.0
10109    17075.0
10110    17075.0
10111        NaN
Name: Y, dtype: float64


,ndatetime,open,high,low,close,volume,Y
0,2019-03-20 15:00:00.000000,0.227123,0.223828,0.229712,10514,12000,10513.0
1,2019-03-20 19:35:12.000000,0.226401,0.222899,0.228994,10513,12000,10488.0
2,2019-03-20 23:01:53.000000,0.226298,0.222280,0.226942,10488,12000,10542.0
3,2019-03-21 01:21:39.000300,0.223722,0.225377,0.227660,10542,12000,10506.0
4,2019-03-21 02:30:19.000300,0.229287,0.226925,0.229096,10506,12000,10501.0


In [4]:
def train_windows(df, ref_day, predict_day):
    X_train, Y_train = [], []
    for i in range(df.shape[0]-predict_day-ref_day+1):
        X_train.append(np.array(df.iloc[i:i+ref_day,1:5]))
        Y_train.append(np.array(df.iloc[i+ref_day:i+ref_day+predict_day]["Y"]))
    return np.array(X_train), np.array(Y_train)

In [5]:
stock = Candledf[:Candledf.shape[0]-300]
test = Candledf[-300:]
test = test.reset_index(drop=True)
stock = stock.reset_index(drop=True)

In [6]:
stock = stock[['open','high','low','close','Y']]
test = test[['open','high','low','close','Y']]
stock.tail()

,open,high,low,close,Y
9807,0.864386,0.862482,0.857392,16632,16631.0
9808,0.857069,0.856907,0.856366,16631,16618.0
9809,0.856760,0.854429,0.853904,16618,16627.0
9810,0.855627,0.853603,0.855648,16627,16622.0
9811,0.855523,0.854326,0.851031,16622,16608.0


In [7]:
X_train, Y_train = train_windows(stock, 12, 1)
X_test, Y_test = train_windows(test, 12, 1)
print(X_train.shape)
print(Y_train.shape)

(9800, 12, 3)
(9800, 1)


In [8]:
class stock_lstm(tf.keras.Model):
  def __init__(self,units):
      super(stock_lstm, self).__init__()
      self.rnn = tf.keras.Sequential([tf.keras.layers.LSTM(units,input_shape=(12,3),dropout=0.2,return_sequences=True,unroll=True),
          tf.keras.layers.LSTM(units,dropout=0.2,unroll=True)])
      self.fc = tf.keras.layers.Dense(50)
      self.out = tf.keras.layers.Dense(1)
  def call(self, inputs, training=None):
      x = self.rnn(inputs)
      x = self.fc(x)
      out = self.out(x)
      return out

In [9]:
batchsz = 12

data = tf.data.Dataset.from_tensor_slices((X_train,Y_train))
data = data.batch(batchsz,drop_remainder=True)
# list(data.as_numpy_iterator())

data_test = tf.data.Dataset.from_tensor_slices((X_test,Y_test))
data_test = data_test.batch(batchsz,drop_remainder=True)
# list(data_test.as_numpy_iterator())


In [10]:
units = 50
epochs = 10


rnn_model = stock_lstm(units)
rnn_model.compile(optimizer  =tf.keras.optimizers.Adam(),loss="mean_absolute_error",metrics=['mean_absolute_error'])
rnn_model.fit(data,epochs=epochs, validation_data = data_test)

Epoch 1/10


InternalError:  Attempting to perform BLAS operation using StreamExecutor without BLAS support
	 [[node stock_lstm/sequential/lstm/lstm_cell/MatMul_14 (defined at \Temp/ipykernel_12388/822064182.py:9) ]] [Op:__inference_train_function_3627]

Function call stack:
train_function
